In [2]:
import gpytorch.kernels as kernel
import torch
from src.test_statistics import *
from src.data import *
import random
import numpy as np
from src.test import *

In [3]:
X_ker = kernel.RBFKernel()
Y_ker = kernel.RBFKernel()

In [4]:
X_ker

In [5]:
n=100
X = torch.randn((n,2))
Y = torch.randn(n)
T = (torch.randn((n))>0).float()

In [6]:
a =torch.tensor([1,1],dtype=float) 

In [7]:
data_train = Data_object(X,Y,T)
data_train_flip = Data_object(X,Y,1-T)

In [8]:
X = torch.randn((n,2))
Y = torch.randn(n)
T = (torch.randn((n))>0).float()

In [9]:
Y1 = torch.randn((1))

In [10]:
data_test = Data_object(X,Y,T)
data_test_flip = Data_object(X,Y,1-T)

In [11]:
W0 = get_W_matrix(X_ker(data_train.X0).evaluate(),1)
W1 = get_W_matrix(Y_ker(data_train.X1).evaluate(),1)

In [12]:
weights = 1/2*torch.ones(n)

In [13]:
K = ker(X_ker)
L = ker(Y_ker)

In [14]:
c = (data_test.T - weights)/(weights*(1-weights))
c

tensor([ 2., -2., -2., -2.,  2.,  2.,  2., -2., -2.,  2., -2.,  2., -2.,  2.,
        -2., -2., -2.,  2.,  2.,  2., -2.,  2., -2., -2., -2.,  2., -2., -2.,
         2.,  2.,  2.,  2., -2.,  2., -2.,  2.,  2.,  2., -2., -2.,  2., -2.,
        -2.,  2.,  2.,  2.,  2., -2., -2.,  2.,  2., -2., -2.,  2.,  2.,  2.,
         2., -2., -2.,  2., -2.,  2., -2., -2., -2., -2.,  2., -2.,  2., -2.,
         2., -2., -2.,  2., -2.,  2.,  2., -2., -2., -2.,  2.,  2.,  2., -2.,
         2.,  2., -2.,  2.,  2., -2., -2., -2.,  2.,  2.,  2.,  2.,  2., -2.,
         2., -2.])

In [15]:
data_train.Y.shape

torch.Size([100])

In [16]:
L(data_train.Y0,data_train.Y).shape

torch.Size([45, 100])

In [17]:
DATE_test_stat(data_train,data_test,X_ker,Y_ker,weights,W0,W1)

0.03502348065376282

In [18]:
DETT_test_stat(data_train,data_test,X_ker,Y_ker,weights,W1)

0.03499886021018028

In [19]:
from sklearn.linear_model import LogisticRegression

In [20]:
weights_model = LogisticRegression()

In [21]:
weights_model.fit(data_train.X,data_train.T)
weights_train = weights_model.predict_proba(data_train.X)[:,1]
weights_test = weights_model.predict_proba(data_test.X)[:,1]

In [22]:
result_dict = kernel_permutation_test(data_train,data_test,X_ker,Y_ker,weights_model,test_stat="DETT",n_bins=10,n_permutations=100,reg=1)

In [23]:
g = lambda X: X[:,0]**2 + X[:,1] 

mu = torch.tensor([0,0.5])
sigma = "ID"
g_0 = g
g_1 = g
noise = 0.1
n_sample =100

In [24]:
a = shift_data_simulation(mu,sigma,g_0,g_1,noise,n_sample)

/Users/jakefawkes/Python/DR_distributional_test/src/data.py:22: UserWarning: To copy construct from a tensor, it is recommended to use sourceTensor.clone().detach() or sourceTensor.clone().detach().requires_grad_(True), rather than torch.tensor(sourceTensor).
  mu = torch.tensor(mu)


In [25]:
import yaml
import src.data as data

In [26]:
f = getattr(data,"shift_data_simulation")

In [27]:
with open("/Users/jakefawkes/Python/DR_distributional_test/config/run_simulated_experiment.yaml", "r") as f:
    cfg = yaml.safe_load(f)


In [28]:
cfg

{'experiment': {'n_itertations': 20,
  'test_stat': ['DATE', 'DETT'],
  'weights_model': 'Logistic_Regression',
  'X_ker': 'RBF',
  'Y_ker': 'RBF'},
 'data': {'generator': 'shift_data_simulation',
  'n_train_sample': 100,
  'n_test_sample': 100,
  'arguments': {'mu': [0.2, 0], 'sigma': 'ID', 'noise': 0.1},
  'functions': {'g_0': 'f_0', 'g_1': 'f_0'}},
 'evaluation': {'n_test': 5000, 'seed': 42}}

In [29]:
def make_data(cfg):
    if cfg["data"]["generator"] == "shift_data_simulation":
        function_dict = {}
        for key in cfg["data"]["functions"]:
            function_dict[key] = getattr(data,cfg["data"]["functions"][key])
    data_train = shift_data_simulation(n_sample=cfg["data"]["n_train_sample"],**function_dict,**cfg["data"]["arguments"])
    data_test = shift_data_simulation(n_sample=cfg["data"]["n_test_sample"],**function_dict,**cfg["data"]["arguments"])
    return data_train,data_test

In [30]:
make_data(cfg)

(<src.data.Data_object at 0x7fd8c9e38f10>,
 <src.data.Data_object at 0x7fd8c50e8250>)

In [31]:
function_dict = {}
for key in cfg["data"]["functions"]:
    function_dict[key] = getattr(data,cfg["data"]["functions"][key])

In [32]:
a = shift_data_simulation(mu,sigma,g_0,g_1,noise,n_sample)

In [33]:
shift_data_simulation(n_sample=cfg["data"]["n_train_sample"],**function_dict,**cfg["data"]["arguments"])

In [34]:
float(True)

1.0